# Fluency Rateを線形重回帰、lasso回帰

In [65]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from statistics import mean

%matplotlib inline

In [10]:
data1 = pd.ExcelFile('main_data_1.xlsx')  #Quantity features and fluency score are in this excel file
df = data1.parse('Sheet1') #Explantion for each coloumn is in Sheet explantion
y = df.filter(["FR"]) #Fluency rating
rates = y['FR'].to_numpy()

In [11]:
def average_maximum_possibility(post):
    return np.mean(np.max(post, axis=1))

def average_post(post):
    return np.mean(post, axis=0)

def kl_divergence(p, q):
    return np.sum(np.where(p != 0, p * np.log(p / q), 0))

def kl_divergence_between_fluent(fluent_posts, target_posts):
    kl_divs = []
    for post in target_posts:
        kl_div = [kl_divergence(post, fluent_posts[i]) for i in range(len(fluent_posts))]
        kl_divs.append(kl_div)
    return np.array(kl_divs)

In [81]:
def compute_quality_features(wsj = 'clean_wsj', n_clusters = 50, native_only = False, print_fluent=True):
    df1 = df.copy()
    
    avg_max_pos = []
    avg_posts = []

    for uttid in range(1, 101):
        post_file_path = 'post/dnn5b_pretrain-dbn_dnn_2000_noise_aug/{}/{}/UTT{}.csv'.format(wsj, n_clusters, uttid)
        post_pd = pd.read_csv(post_file_path, header=None)
        post = post_pd.to_numpy()
        avg_max_pos.append(average_maximum_possibility(post))
        avg_posts.append(average_post(post))

    df1['average_maximum_possibility'] = avg_max_pos
    avg_posts = np.array(avg_posts)

    fluent_speaker_idx = np.arange(-10, 1) if native_only else np.where(rates >= 8.0)
    if print_fluent:
        print('Fluent speakers: ', fluent_speaker_idx)
    fluent_posts = avg_posts[fluent_speaker_idx]

    kl_divs_fluent = kl_divergence_between_fluent(fluent_posts, avg_posts)
    kl_divs_fluent_mean = np.mean(kl_divs_fluent, axis=1)

    df1['average_KLdivergence_with_fluent'] = kl_divs_fluent_mean
    return df1

In [70]:
def elastic_cv(X,y,k,c):
    coef = []
    r2_train = []
    r2_test = []
    r2 = []
    all_pred = []
    all_y = []
    kf = KFold(n_splits = k, shuffle = True)
    clf = ElasticNet(alpha=0.1,l1_ratio=0.9)
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        ss = StandardScaler()
        X_train = ss.fit_transform(X_train)
        X_test = ss.transform(X_test)
        clf.fit(X_train, y_train) 
        coef.append(clf.coef_)
        pred_test = clf.predict(X_test)
        r2_train.append(r2_score(y_train, clf.predict(X_train)))
        r2_test.append(r2_score(y_test, pred_test))
        all_pred.append(pred_test)
        all_y.append(y_test.values)
        
    coef_mean = []
    for j in range(c):
        coef_j = []
        for l in range(k):
            coef_j.append(coef[l][j])
        coef_mean.append(mean(coef_j))
    
    from scipy.stats import pearsonr
  
    correlation, p_value = pearsonr(np.ravel(all_pred), np.ravel(all_y))
    
    return coef_mean, correlation, p_value, mean(r2_test)

def train(X, y, k, c, n_trials = 10, print_current=True):
    coef_list = np.zeros((c))
    corr_list = []
    p_list = []
    r2_list = []
    for i in range(10):
        coef, correlation, p_value, r2_test = elastic_cv(X, y, k, c)
        coef_list += np.array(coef)
        corr_list.append(correlation)
        p_list.append(p_value)
        r2_list.append(r2_test)

    coef_mean = coef_list / n_trials
    correlation = mean(corr_list)
    p_value = mean(p_list)
    r2_test = mean(r2_list)

    if print_current:
        print("Average 回帰変数", coef_mean)
        print("Average 相関係数:", correlation)
        print("Average p値:", p_value)
        print('Average r2_test:', r2_test)

    return coef_mean, correlation, p_value, r2_test

#### Original results

In [36]:
X_ori = df.filter(["ATCL_P","phonation_rate"])
_ = train(X_ori, y, 5, 2, n_trials=10)

Average 回帰変数 [0.76823905 1.33191602]
Average 相関係数: 0.8193963724844843
Average p値: 4.31705778494182e-25
Average r2_test: 0.6652608144851501


#### clean wsj 50 MAX only

In [55]:
post_df = compute_quality_features(wsj = 'clean_wsj', n_clusters=50)
X_max = post_df.filter(["ATCL_P","phonation_rate", "average_maximum_possibility"])
_ = train(X_max, y, 5, 3, n_trials=10)

Fluent speakers:  (array([42, 45, 49, 52, 68, 72, 78, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99],
      dtype=int64),)
Average 回帰変数 [0.74965062 1.38998177 0.43771613]
Average 相関係数: 0.8513049136968747
Average p値: 2.059881298710742e-28
Average r2_test: 0.718934100280917


#### clean wsj 50 KL only, native + fluent

In [69]:
X_native_fluent = post_df.filter(["ATCL_P","phonation_rate", \
    "average_KLdivergence_with_fluent"])
_ = train(X_native_fluent, y, 5, 3, n_trials=10)

Average 回帰変数 [ 0.57673893  0.61764616 -1.05979258]
Average 相関係数: 0.8844118324671238
Average p値: 7.384592324730762e-34
Average r2_test: 0.7714799577586595


In [84]:
kl_list = post_df.filter(['average_KLdivergence_with_fluent']).to_numpy()
print("Unfluent speaker average kl divergence", np.mean(kl_list[np.where(rates < 8.0)]))
print("Fluent speaker average kl divergence", np.mean(kl_list[np.where(rates >= 8.0)]))

Unfluent speaker average kl divergence 0.269489501696798
Fluent speaker average kl divergence 0.17976784672322937


#### clean wsj 50 KL only, native only

In [68]:
native_only_df = compute_quality_features(wsj = 'clean_wsj', n_clusters=50, native_only=True)
X_native_only = native_only_df.filter(["ATCL_P","phonation_rate", \
    "average_KLdivergence_with_fluent"])
_ = train(X_native_only, y, 5, 3, n_trials=10)

Fluent speakers:  [-10  -9  -8  -7  -6  -5  -4  -3  -2  -1   0]
Average 回帰変数 [ 0.54968446  0.75011326 -0.96225617]
Average 相関係数: 0.8810404283877226
Average p値: 6.604284407792609e-33
Average r2_test: 0.7546854116673984


#### clean wsj 50 full

In [78]:
X_curr = post_df.filter(["ATCL_P","phonation_rate", "average_maximum_possibility", \
    "average_KLdivergence_with_fluent"])
_ = train(X_curr, y, 5, 4, n_trials=10)

Average 回帰変数 [ 0.57126856  0.70016824  0.3822208  -1.01325073]
Average 相関係数: 0.912567102649622
Average p値: 1.9576403861146702e-39
Average r2_test: 0.8189879096917841


#### Benchmark clusters

In [ ]:
cluster_list = [50, 200, 500, 1000, 2000]
corr_list = []
r2_list = []
for n in cluster_list:
    curr_df = compute_quality_features(wsj='clean_wsj', n_clusters=n, print_fluent=False)
    X_curr = curr_df.filter(["ATCL_P","phonation_rate", "average_maximum_possibility",\
         "average_KLdivergence_with_fluent"])
    _, corr, _, r2_test = train(X_curr, y, 5, 4, n_trials=10, print_current=False)
    corr_list.append(corr)
    r2_list.append(r2_test)
cluster_df = pd.DataFrame({'correlation': corr_list, 'r2_score': r2_list}).set_axis(cluster_list)
cluster_df.sort_values(by=['correlation'], ascending=False)

#### Benchmark record length

In [82]:
wsj_list = ['clean_wsj', 'clean_wsj_4', 'clean_wsj_6', 'clean_wsj_8']
corr_list = []
r2_list = []
for wsj in wsj_list:
    curr_df = compute_quality_features(wsj=wsj, n_clusters=50, print_fluent=False)
    X_curr = curr_df.filter(["ATCL_P","phonation_rate", "average_maximum_possibility",\
         "average_KLdivergence_with_fluent"])
    _, corr, _, r2_test = train(X_curr, y, 5, 4, n_trials=10, print_current=False)
    corr_list.append(corr)
    r2_list.append(r2_test)
length_df = pd.DataFrame({'correlation': corr_list, 'r2_score': r2_list}).set_axis(wsj_list)
length_df.sort_values(by=['correlation'], ascending=False)

,correlation,r2_score
clean_wsj,0.913351,0.818371
clean_wsj_8,0.910369,0.807543
clean_wsj_6,0.899786,0.794141
clean_wsj_4,0.891883,0.772022
